# 1. Build Team Identities, Company Employment and User Identities

Notebook to build all of the different identities assumed available when I have initiailised Type Refinery. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We will take the Type Refinery User as an Example of the Process of making a group of Complex Identity using the official [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/tree/main/extension-definition-specifications/identity-contact-information) specification, and combining it with the standard Stix approach. There are three stages:

1. Create the `User_Account` first,
2. Then the `Email_Address` objects, attaching the new User_Account.["id"]
3. Then create the `EmailContact`, `SocialMediaContact`, `ContactNumber` lists of sub-objects from the specification, attaching their User_Account.["id"] and Email_Address.["id"] as required
4. Create the [Identity Stix object](https://docs.oasis-open.org/cti/stix/v2.1/os/stix-v2.1-os.html#_wh296fiwpklp) with the [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/identity-contact-information/Identity%20Contact%20Information.docx), and joining the sub objects to the Extension object and then down to the base Identity object
5. Collect all of the objects and put them into a bundle, and save into a json

Then we will create identities fro the three dummy users that are always clicking the wrong links, but at least they are good on reporting. Finally, we will employ the four Identities with our current company, and save the complete group of objects into a bundle

Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (It also verifies the objects are created correctly, and is cool).

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

## A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.User_Account.make_user_account import main as make_user_account
from Block_Families.Objects.SCO.Email_Addr.make_email_addr import main as make_email_addr
from Block_Families.Objects.SDO.Identity.make_identity import main as make_identity
from utilities import emulate_ports, unwind_ports

path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
identities_data = [
    {
        "who": "me", "email": "SCO/Email_Addr/email_addr_TR_user.json", 
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "TR_Identity"
    }, 
    {
        "who": "usr1", "email": "SCO/Email_Addr/email_addr_IT_user1.json", 
        "acct": "SCO/User_Account/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "Usr1"
    }, 
    {
        "who": "usr2", "email": "SCO/Email_Addr/email_addr_IT_user2.json", 
        "acct": "SCO/User_Account/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "Usr2"
    }, 
    {
        "who": "usr3", "email": "SCO/Email_Addr/email_addr_IT_user3.json", 
        "acct": "SCO/User_Account/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "Usr3"
    }, 
]
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "employed"
}
# Make the Variable to hold the Bundle List
bundle_list = []
results_paths = {}

## B. Make the User of this Type Refinery

once on initialisation, otherwise you must update the identity object

### B.1 First Select the User Data

In [4]:
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]


### B.2 Make the User Account
Simply pass the paths for the input file and where the results should be written

In [5]:
def local_make_acct(acct_path, results_path):
    # Set the Relative Input and Output Paths for the block
    acct_data_rel_path =  path_base + acct_path
    acct_results_rel_path = results_base + results_path + "__acct.json"
    results_paths["acct"] = acct_results_rel_path
    # Run the Make User Account block
    make_user_account(acct_data_rel_path,acct_results_rel_path)
    # Retrieve the saved file
    if os.path.exists(acct_results_rel_path):
        with open(acct_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["user-account"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            usr_acct = UserAccount(**stix_object)
            print(usr_acct.serialize(pretty=True))
            bundle_list.append(usr_acct)

# make the account object
local_make_acct(acct_path, results_path)

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}


### B.3 Make the Email Address

In [6]:
def local_make_email(email_path, results_path, acct_results):
    #
    # 1. Set the Relative Input and Output Paths for the block
    #
    email_data_rel_path = path_base + email_path
    email_results_rel_path = results_base + results_path + "__email.json"
    results_paths['email'] = email_results_rel_path
    #
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    emulate_ports(email_data_rel_path, [acct_results])
    #
    # Make the Email Address object
    #
    make_email_addr(email_data_rel_path,email_results_rel_path)
    #
    # Remove Port Emulation - Fix the data file so it only has form data
    #
    unwind_ports(email_data_rel_path)
    # Retrieve the saved file
    if os.path.exists(email_results_rel_path):
        with open(email_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["email-addr"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            email_addr = EmailAddress(**stix_object)
            print(email_addr.serialize(pretty=True))
            bundle_list.append(email_addr)



# make the email object
email_results_rel_path=""
local_make_email(email_path, results_path, results_paths['acct'])

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}


### B.4 Make the Identity

In [7]:
def local_make_identity(ident_path, results_path, email_results, acct_results):
    # Set the Relative Input and Output Paths for the block
    identity_data_rel_path = path_base + ident_path
    identity_results_rel_path = results_base + results_path + "__ident.json"
    #
    # NOTE: This code is only To fake input ports
    # Add the USer Account object and the  EmailAddress
    #  to the Identity Form object
    ##
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    emulate_ports(identity_data_rel_path, [acct_results, email_results])
    # Make the Email Address object
    make_identity(identity_data_rel_path,identity_results_rel_path)
    #
    # Remove Port Emulation - Fix the data file so it only has form data
    #
    unwind_ports(identity_data_rel_path)
    # Retrieve the saved file
    if os.path.exists(identity_results_rel_path):
        with open(identity_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["identity"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            identity = Identity(**stix_object)
            print(identity.serialize(pretty=True))
            bundle_list.append(identity)



# make the email object
local_make_identity(ident_path, results_path, email_results=results_paths['email'], acct_results=results_paths['acct'])

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--7580544f-0671-4633-b9a2-16fef9bcf124",
    "created": "2023-12-12T14:22:17.352739Z",
    "modified": "2023-12-12T14:22:17.352739Z",
    "name": "Trusty Jones",
    "description": "A Trusty Individual",
    "roles": [
        "soc",
        "support"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "contact_numbers": [
                {
                    "contact_number_type": "work-phone",
                    "contact_number": "0418-208-368"
                }
            ],
            "email_addresses": [
                {
                    "digital_contact_type": "work",
                    "email_address_ref": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3"
                }
            ],
            "first_nam